In [1]:
import pandas as pd
import re

df = pd.read_csv("./dataset/PriceList.csv")
df.head()

,COMP_TYPE,BRAND_NAME,PACK_NAME,PRD_CD,PRD_DESC1,CONV_FACTOR,MRP_UN,RP,DEALER_MARGIN,Textbox51,RP1
0,BEV,7 UP,1.25L,1379,7 UP 1.25 L PET 12 Rs65,12,65.0,60.0000,8.33,0.0,60.0000
1,BEV,7 UP,1.25L,1894,7 UP 1.25 L PET 15 Rs50 PROMO,15,50.0,46.6667,7.14,0.0,46.6667
2,BEV,7 UP,1.25L,2274,7 UP 1.25 L PET 12 Rs50,12,50.0,45.0000,11.11,0.0,45.0000
3,BEV,7 UP,1.25L,2282,7 UP 1.25 L PET 12 Rs52,12,52.0,47.5000,9.47,0.0,47.5000
4,BEV,7 UP,1.25L,2288,7 UP 1.25 L PET 12 Rs50 PROMO,12,50.0,45.8333,9.09,0.0,45.8333


In [27]:
def extract_product_name(description):
    # Match the first part of the description that doesn't include digits, ML or L
    match = re.match(r"([^\d]+)", description)
    return match.group().strip() if match else None

# Define a function to extract the category from the description
def extract_category(description):
    # Match PET, CAN, TETRA, etc. in the description
    match = re.search(r"(PET|CAN|TETRA|RGP)", description)
    return match.group().strip() if match else None


def extract_quantity_and_unit(pack_name):
    # Use regular expression to find the quantity and unit
    match = re.search(r'(\d+\.?\d*)(ML|L|ml)', pack_name.upper())
    if match:
        # Extract quantity and unit from the match groups
        quantity = match.group(1)
        unit = match.group(2)
        if unit=="ML":
            unit = unit.lower()
        return float(quantity), str(unit)
    else:
        # Return None if no match is found
        return None, None


In [28]:
df1 = pd.DataFrame()

df1['product_id'] = df['PRD_CD']
df1['product_name'] = df['PRD_DESC1'].apply(extract_product_name)
df1['piecepercase'] = df['CONV_FACTOR']
df1['category'] = df['PRD_DESC1'].apply(extract_category)
df1['price'] = df['MRP_UN']
df1[['quantity', 'unit']] = df['PACK_NAME'].apply(
    lambda x: pd.Series(extract_quantity_and_unit(x))
)


df1.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,None,12,PET,65.0,1.25,L
1,1894,None,15,PET,50.0,1.25,L
2,2274,None,12,PET,50.0,1.25,L
3,2282,None,12,PET,52.0,1.25,L
4,2288,None,12,PET,50.0,1.25,L


In [29]:
df1.tail()

,product_id,product_name,piecepercase,category,price,quantity,unit
1071,8308,TROP PR APPLE DELGT,24,PET,40.0,500.0,ml
1072,8310,TROP PR LICHI DELIT,24,PET,40.0,500.0,ml
1073,8549,TROP PR MIX FRUIT DEL,24,PET,40.0,500.0,ml
1074,8552,TROP PR MIX FRUIT DEL,24,PET,30.0,500.0,ml
1075,3375,TROP SLICE ALPHONSO,24,PET,50.0,600.0,ml


In [30]:
df1["product_name"] = df1["product_name"].fillna("7 UP")

In [31]:
df1.isna().sum()

product_id       0
product_name     0
piecepercase     0
category        53
price            0
quantity        15
unit            15
dtype: int64

In [34]:
df1.dropna(inplace=True)
df1.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,7 UP,12,PET,65.0,1.25,L
1,1894,7 UP,15,PET,50.0,1.25,L
2,2274,7 UP,12,PET,50.0,1.25,L
3,2282,7 UP,12,PET,52.0,1.25,L
4,2288,7 UP,12,PET,50.0,1.25,L


In [36]:
df1.shape

(1023, 7)

In [37]:
df1.to_csv("ProductPreprocessed.csv",index=False)

In [41]:
data = pd.read_csv("ProductPreprocessed.csv")

In [42]:
data.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,7 UP,12,PET,65.0,1.25,L
1,1894,7 UP,15,PET,50.0,1.25,L
2,2274,7 UP,12,PET,50.0,1.25,L
3,2282,7 UP,12,PET,52.0,1.25,L
4,2288,7 UP,12,PET,50.0,1.25,L


In [45]:
data_unique = data.drop_duplicates(subset=['product_name', 'category', 'quantity', 'unit'])

In [47]:
data_unique.shape

(322, 7)

In [48]:
data_unique.to_csv('ProdcutsDupRemoved.csv', index=False)

## FuzzyLogic

In [51]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

d:\python\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [63]:
dup = pd.read_csv("ProdcutsDupRemoved.csv")
dup.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,7 UP,12,PET,65.0,1.25,L
1,1376,7 UP,12,PET,65.0,1.50,L
2,3585,7 UP,24,CAN,20.0,150.00,ml
3,4787,7 UP,30,CAN,30.0,180.00,ml
4,2211,7 UP,12,PET,35.0,1.00,L


In [66]:

# List of all possible categories
categories = ["ORANGE", "APPLE", "LYCHEE", "POMEGRANATE", "MIX FRUIT", "GUAVA"]

# Function to find the best matching category for a given input
def find_best_match(input_str):
    # Initialize variables to store the best match and its score
    best_match = None
    best_score = -1
    
    # Iterate through categories to find the best match
    for category in categories:
        score = fuzz.ratio(input_str, category)
        if score > best_score:
            best_score = score
            best_match = category
    
    # If the best score is below a certain threshold, consider it a misspelling
    if best_score < 20:
        return None
    else:
        return best_match

# Given list of values
values = dup.product_name.values

# Transform the values
transformed_values = []

for value in values:
    if "TROP" in value:
        # Split the value by space
        parts = value.split()
        # Find the best match for the first part (e.g., "TROPICANA")
        subset = find_best_match(value)
        transformed_values.append(f"TROPICANA {subset}")
    else:
        transformed_values.append(value)

dup["transformed_values"] = transformed_values


In [67]:
dup.tail()

,product_id,product_name,piecepercase,category,price,quantity,unit,transformed_values
317,3692,TROP PR APPLE DELIGT,24,PET,40.0,500.0,ml,TROPICANA APPLE
318,8307,TROP PR GUAVA DELGT,24,PET,40.0,500.0,ml,TROPICANA GUAVA
319,8308,TROP PR APPLE DELGT,24,PET,40.0,500.0,ml,TROPICANA APPLE
320,8310,TROP PR LICHI DELIT,24,PET,40.0,500.0,ml,TROPICANA APPLE
321,3375,TROP SLICE ALPHONSO,24,PET,50.0,600.0,ml,TROPICANA APPLE


In [69]:
dup.to_csv("Proddup2.csv",index=False)

In [70]:
dup2 = pd.read_csv("Proddup2.csv")
dup2.head()

,product_id,product_name,piecepercase,category,price,quantity,unit,transformed_values
0,1379,7 UP,12,PET,65.0,1.25,L,7 UP
1,1376,7 UP,12,PET,65.0,1.50,L,7 UP
2,3585,7 UP,24,CAN,20.0,150.00,ml,7 UP
3,4787,7 UP,30,CAN,30.0,180.00,ml,7 UP
4,2211,7 UP,12,PET,35.0,1.00,L,7 UP


In [71]:
dup2["product_name"] = dup2["transformed_values"]

In [72]:
dup2.drop(["transformed_values"],axis=1,inplace=True)
dup2.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,7 UP,12,PET,65.0,1.25,L
1,1376,7 UP,12,PET,65.0,1.50,L
2,3585,7 UP,24,CAN,20.0,150.00,ml
3,4787,7 UP,30,CAN,30.0,180.00,ml
4,2211,7 UP,12,PET,35.0,1.00,L


In [73]:
data_unique = dup2.drop_duplicates(subset=['product_name', 'category', 'quantity', 'unit'])

In [74]:
data_unique.shape

(198, 7)

In [75]:
data_unique.to_csv("Proddup3.csv",index=False)

In [76]:
final = pd.read_csv("Proddup3.csv")
final.head()

,product_id,product_name,piecepercase,category,price,quantity,unit
0,1379,7 UP,12,PET,65.0,1.25,L
1,1376,7 UP,12,PET,65.0,1.50,L
2,3585,7 UP,24,CAN,20.0,150.00,ml
3,4787,7 UP,30,CAN,30.0,180.00,ml
4,2211,7 UP,12,PET,35.0,1.00,L


In [78]:
def calculate_sgst_cgst_cess(product_name):
    if "AQUAFINA" in product_name:
        return (9, 9, 0)  # SGST, CGST, CESS
    elif "TROPICANA" in product_name or "SLICE" in product_name:
        return (6, 6, 0)  # SGST, CGST, CESS
    else:
        return (7, 7, 12)  # SGST, CGST, CESS

# Apply the function to create new columns
final['SGST'], final['CGST'], final['CESS'] = zip(*final['product_name'].apply(calculate_sgst_cgst_cess))

final.tail()


,product_id,product_name,piecepercase,category,price,quantity,unit,SGST,CGST,CESS
189,8421,TROPICANA POMEGRANATE,12,TETRA,110.0,1.0,L,6,6,0
190,3651,TROPICANA POMEGRANATE,30,TETRA,20.0,200.0,ml,6,6,0
191,8318,TROPICANA POMEGRANATE,30,PET,20.0,180.0,ml,6,6,0
192,3689,TROPICANA POMEGRANATE,24,PET,40.0,500.0,ml,6,6,0
193,4268,TROPICANA POMEGRANATE,30,PET,20.0,250.0,ml,6,6,0


In [2]:
final.to_csv("FinalProducts.csv",index=False)

NameError: name 'final' is not defined

In [3]:
final = pd.read_csv("FinalProductsUpdated.csv")
final.head()

,S.NO,product_id,product_name,piece per case,category,MRP price,quantity,unit,SGST,CGST,CESS
0,1,2977,7 UP,12,PET,60,1.25,L,7,7,12
1,2,4352,7 UP,9,PET,99,2.25,L,7,7,12
2,3,1008,7 UP,24,RGP,10,200.00,ml,7,7,12
3,4,3591,7 UP,24,CAN,35,250.00,ml,7,7,12
4,5,1961,7 UP,30,PET,20,250.00,ml,7,7,12


In [7]:
final = final.rename(columns={"MRP price":"price","piece per case":"piecepercase"})

In [8]:
df = final.rename(columns={
    'product_id': 'p_id',
    'piecepercase': 'piecesPerCase',
    'product_name': 'productName',
    'SGST': 'SGST',
    'CGST': 'CGST',
    'CESS': 'CESS',
})

new_df = df[['p_id', 'productName', 'piecesPerCase', 'category', 'price', 'quantity', 'unit', 'SGST', 'CGST', 'CESS']].copy()

# Convert the DataFrame to a list of dictionaries with 'taxInfo' as a separate object
products_list = new_df.to_dict(orient='records')

{'p_id': 1379,
 'productName': '7 UP',
 'piecesPerCase': 12,
 'category': 'PET',
 'price': 65.0,
 'quantity': 1.25,
 'unit': 'L',
 'taxInfo': {'SGST': 7, 'CGST': 7, 'CESS': 12}}

In [8]:
for p in products_list:
    p["productName"] = f"{p['productName']} {p['category']} {p['quantity']} {p['unit']}"

In [9]:
products_list

[{'p_id': 2977,
  'productName': '7 UP',
  'piecesPerCase': 12,
  'category': 'PET',
  'price': 60,
  'quantity': 1.25,
  'unit': 'L',
  'SGST': 7,
  'CGST': 7,
  'CESS': 12},
 {'p_id': 4352,
  'productName': '7 UP',
  'piecesPerCase': 9,
  'category': 'PET',
  'price': 99,
  'quantity': 2.25,
  'unit': 'L',
  'SGST': 7,
  'CGST': 7,
  'CESS': 12},
 {'p_id': 1008,
  'productName': '7 UP',
  'piecesPerCase': 24,
  'category': 'RGP',
  'price': 10,
  'quantity': 200.0,
  'unit': 'ml',
  'SGST': 7,
  'CGST': 7,
  'CESS': 12},
 {'p_id': 3591,
  'productName': '7 UP',
  'piecesPerCase': 24,
  'category': 'CAN',
  'price': 35,
  'quantity': 250.0,
  'unit': 'ml',
  'SGST': 7,
  'CGST': 7,
  'CESS': 12},
 {'p_id': 1961,
  'productName': '7 UP',
  'piecesPerCase': 30,
  'category': 'PET',
  'price': 20,
  'quantity': 250.0,
  'unit': 'ml',
  'SGST': 7,
  'CGST': 7,
  'CESS': 12},
 {'p_id': 4949,
  'productName': '7 UP',
  'piecesPerCase': 24,
  'category': 'PET',
  'price': 40,
  'quantity': 7

In [86]:
import json

# Assuming you have 'products_list' containing your data

# Specify the file path where you want to save the JSON file
file_path = 'products.json'

# Save 'products_list' to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(products_list, json_file, indent=4)
